In [1]:
# CNN
#Model Training

In [2]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import random, shutil
from keras.models import Sequential
from keras.layers import (
    Dropout,
    Conv2D,
    Flatten,
    Dense,
    MaxPooling2D,
    BatchNormalization,
)
from keras.models import load_model


def generator(
    dir,
    gen=image.ImageDataGenerator(rescale=1.0 / 255),
    shuffle=True,
    batch_size=1,
    target_size=(24, 24),
    class_mode="categorical",
):

    return gen.flow_from_directory(
        dir,
        batch_size=batch_size,
        shuffle=shuffle,
        color_mode="grayscale",
        class_mode=class_mode,
        target_size=target_size,
    )


BS = 32
TS = (24, 24)
train_batch = generator(r"C:\Users\HarsH\Downloads\Compressed\archive\dataset_new\train", shuffle=True, batch_size=BS, target_size=TS)
valid_batch = generator(r"C:\Users\HarsH\Downloads\Compressed\archive\dataset_new\test", shuffle=True, batch_size=BS, target_size=TS)
SPE = len(train_batch.classes) // BS
VS = len(valid_batch.classes) // BS
print(SPE, VS)

img, labels = next(train_batch)
print(img.shape)

cnn_model = Sequential(
    [
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(24, 24, 1)),
        MaxPooling2D(pool_size=(1, 1)),
        Conv2D(32, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(1, 1)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(1, 1)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(4, activation="softmax"),
    ]
)
cnn_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
cnn_model.fit(
    train_batch,
    validation_data=valid_batch,
    epochs=50,
    steps_per_epoch=SPE,
    validation_steps=VS,
)
cnn_model.save("models/cnnCat2.h5", overwrite=True)

Found 2467 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
77 13
(32, 24, 24, 1)
Epoch 1/50
77/77 [==============================] - 10s 125ms/step - loss: 0.6933 - accuracy: 0.6747 - val_loss: 0.4330 - val_accuracy: 0.7885
Epoch 2/50
77/77 [==============================] - 9s 116ms/step - loss: 0.4139 - accuracy: 0.7869 - val_loss: 0.3805 - val_accuracy: 0.8077
Epoch 3/50
77/77 [==============================] - 9s 120ms/step - loss: 0.3809 - accuracy: 0.8057 - val_loss: 0.3541 - val_accuracy: 0.8125
Epoch 4/50
77/77 [==============================] - 9s 116ms/step - loss: 0.3501 - accuracy: 0.8316 - val_loss: 0.3146 - val_accuracy: 0.8389
Epoch 5/50
77/77 [==============================] - 9s 115ms/step - loss: 0.3149 - accuracy: 0.8456 - val_loss: 0.3019 - val_accuracy: 0.8534
Epoch 6/50
77/77 [==============================] - 9s 117ms/step - loss: 0.2746 - accuracy: 0.8669 - val_loss: 0.2843 - val_accuracy: 0.8582
Epoch 7/50
77/77 [========================

C:\Users\HarsH\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#Capture

In [ ]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

mixer.init()
sound = mixer.Sound(r'C:\Users\HarsH\Desktop\Desktop\CNN\alarm.wav')

face = cv2.CascadeClassifier(r'C:\Users\HarsH\Desktop\Desktop\CNN\haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(r'C:\Users\HarsH\Desktop\Desktop\CNN\haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(r'C:\Users\HarsH\Desktop\Desktop\CNN\haar cascade files\haarcascade_righteye_2splits.xml')

lbl = ['Close', 'Open']

model = load_model('models/cnncat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count = 0
score = 0
thicc = 2
rpred = [99]
lpred = [99]

while True:
    ret, frame = cap.read()
    height, width = frame.shape[:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25))
    left_eye = reye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (100, 100, 100), 1)

    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y + h, x:x + w]
        count = count + 1
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye, (24, 24))
        r_eye = r_eye / 255
        r_eye = r_eye.reshape(24, 24, -1)
        r_eye = np.expand_dims(r_eye, axis=0)
        rpred = model.predict(r_eye)
        rpred_class = np.argmax(rpred, axis=1)  # Get the class with the highest probability

        if rpred_class[0] == 1:
            lbl = 'Alert'
        if rpred_class[0] == 0:
            lbl = 'Sleepy!'
        break

    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y + h, x:x + w]
        count = count + 1
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye, (24, 24))
        l_eye = l_eye / 255
        l_eye = l_eye.reshape(24, 24, -1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model.predict(l_eye)
        lpred_class = np.argmax(lpred, axis=1)  # Get the class with the highest probability
        
        cv2.putText(frame,'Press "Q" to exit.',(200,height-60),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),
                       thickness=1,lineType=cv2.LINE_AA)
        
        if lpred_class[0] == 1:
            lbl = 'Alert'
        if lpred_class[0] == 0:
            lbl = 'Sleepy!'
        break

    if (rpred_class[0] == 0 or lpred_class[0] == 0):  # Check if either eye predicts "Sleepy"
        score = score + 1
        cv2.putText(frame, "Sleepy!", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        
    else:
        score = score - 1
        cv2.putText(frame, "Alert", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

    if score < 0:
        score = 0
    cv2.putText(frame, 'Score:' + str(score), (100, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    if score > 15:
        cv2.imwrite(os.path.join(path, 'image.jpg'), frame)
        try:
            sound.play()
        except:
            pass
        if thicc < 16:
            thicc = thicc + 2
        else:
            thicc = thicc - 2
            if thicc < 2:
                thicc = 2
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step
